# Model Construction
This notebook is used to construct a model that can predict the number of available bike stands at a station (and thereby also the number of available bikes). In the first section the required data (bike dynamimc data & weather dynamic data) is imported and some derived features are added to make the dataframe suitable for machine learning. The second part uses random forests to create a model for each station and python's pickel library to export the model.

In [1]:
import db_connection as db
import pandas as pd
import datetime as dt

## 1. Data Import and Analysis
We import selected columns of two tables from our database (BikeStationDynamicData & WeatherData). Because we queried Bike Station Dynamic Data every five minutes and Weather only every 30 minutes, we first need to join the two tables in such a way that each entry in the BikeStationDynamicData is paired up with the closest entry in the WeatherData table. This is possible as both tables store a Timestamp (one in seconds, other in milliseconds). 

Then we do some conversions (unix timestamp to datetime) and derive some features which we think might be helpful in training the model. As datetime entry is difficult for a machine learning algorith to interpret, we break the datetime into several features. 

In [2]:
# Set up query
my_query = '''
select Number, B.Timestamp, BikeStands, AvailableStands, W.Timestamp*1000 as WeatherTime, Temperature, WindSpeed, Clouds, Rain, Snow, Sunrise*1000 as Sunrise, Sunset*1000 as Sunset
from BikeStationDynamicData as B
Left Outer Join WeatherData as W ON
W.TimeStamp = (select We.Timestamp 
from WeatherData as We
where We.Timestamp*1000 >= B.Timestamp - 8600000 and We.TimeStamp*1000 < B.Timestamp + 8600000
order by ABS(We.Timestamp*1000 - B.Timestamp)
Limit 1);
'''

# Query database
df = pd.read_sql(my_query, con=db.cnx)    

In [3]:
# check if any null entries
df.isna().sum()

Number                  0
Timestamp               0
BikeStands              0
AvailableStands         0
WeatherTime             0
Temperature             0
WindSpeed               0
Clouds                  0
Rain               364221
Snow               366811
Sunrise                 0
Sunset                  0
dtype: int64

In [4]:
# check if right size
df.shape

(366811, 12)

In [5]:
# assign to a new df and keep original (query takes ages, don't want to repeat in case we need reset)
new_df = df

# convert unix timestamps (ms) to datetime
new_df.Timestamp =pd.to_datetime(df.Timestamp, unit='ms').dt.tz_localize('UTC').dt.tz_convert("Europe/Dublin")
new_df.WeatherTime =pd.to_datetime(df.WeatherTime, unit='ms').dt.tz_localize('UTC').dt.tz_convert("Europe/Dublin")
new_df.Sunrise =(pd.to_datetime(df.Sunrise, unit='ms')).dt.tz_localize('UTC').dt.tz_convert("Europe/Dublin")
new_df.Sunset =(pd.to_datetime(df.Sunset, unit='ms')).dt.tz_localize('UTC').dt.tz_convert("Europe/Dublin")

In [6]:
# add weekday and hour:minute columns
new_df["hourOfDay"] = new_df.Timestamp.apply(lambda time: "{0:0>2}:{1:0>2}".format(time.hour, time.minute))
new_df["dayOfWeek"] = new_df.Timestamp.apply(lambda time: time.weekday())
new_df.head()

,Number,Timestamp,BikeStands,AvailableStands,WeatherTime,Temperature,WindSpeed,Clouds,Rain,Snow,Sunrise,Sunset,hourOfDay,dayOfWeek
0,1,2018-03-27 09:48:06+01:00,31,12,2018-03-27 10:00:00+01:00,281,9.3,75,NaN,None,2018-03-27 07:09:57+01:00,2018-03-27 19:51:46+01:00,09:48,1
1,1,2018-03-27 10:50:37+01:00,31,7,2018-03-27 10:30:00+01:00,281,9.8,75,NaN,None,2018-03-27 07:09:54+01:00,2018-03-27 19:51:48+01:00,10:50,1
2,1,2018-03-27 11:01:16+01:00,31,5,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,2018-03-27 07:09:51+01:00,2018-03-27 19:51:50+01:00,11:01,1
3,1,2018-03-27 11:09:38+01:00,31,4,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,2018-03-27 07:09:51+01:00,2018-03-27 19:51:50+01:00,11:09,1
4,1,2018-03-27 11:10:16+01:00,31,6,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,2018-03-27 07:09:51+01:00,2018-03-27 19:51:50+01:00,11:10,1


In [7]:
# add boolean stating if it is a workday
new_df["workday"] = new_df.dayOfWeek.apply(lambda day: day in range(0, 5))
new_df.head()

,Number,Timestamp,BikeStands,AvailableStands,WeatherTime,Temperature,WindSpeed,Clouds,Rain,Snow,Sunrise,Sunset,hourOfDay,dayOfWeek,workday
0,1,2018-03-27 09:48:06+01:00,31,12,2018-03-27 10:00:00+01:00,281,9.3,75,NaN,None,2018-03-27 07:09:57+01:00,2018-03-27 19:51:46+01:00,09:48,1,True
1,1,2018-03-27 10:50:37+01:00,31,7,2018-03-27 10:30:00+01:00,281,9.8,75,NaN,None,2018-03-27 07:09:54+01:00,2018-03-27 19:51:48+01:00,10:50,1,True
2,1,2018-03-27 11:01:16+01:00,31,5,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,2018-03-27 07:09:51+01:00,2018-03-27 19:51:50+01:00,11:01,1,True
3,1,2018-03-27 11:09:38+01:00,31,4,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,2018-03-27 07:09:51+01:00,2018-03-27 19:51:50+01:00,11:09,1,True
4,1,2018-03-27 11:10:16+01:00,31,6,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,2018-03-27 07:09:51+01:00,2018-03-27 19:51:50+01:00,11:10,1,True


In [8]:
# add boolean stating if it is during rush hours
new_df["rushHour"] = new_df.hourOfDay.apply(lambda time: "07:00" <= time <= "09:00" or "16:00" <= time <= "19:00")
new_df.head()

,Number,Timestamp,BikeStands,AvailableStands,WeatherTime,Temperature,WindSpeed,Clouds,Rain,Snow,Sunrise,Sunset,hourOfDay,dayOfWeek,workday,rushHour
0,1,2018-03-27 09:48:06+01:00,31,12,2018-03-27 10:00:00+01:00,281,9.3,75,NaN,None,2018-03-27 07:09:57+01:00,2018-03-27 19:51:46+01:00,09:48,1,True,False
1,1,2018-03-27 10:50:37+01:00,31,7,2018-03-27 10:30:00+01:00,281,9.8,75,NaN,None,2018-03-27 07:09:54+01:00,2018-03-27 19:51:48+01:00,10:50,1,True,False
2,1,2018-03-27 11:01:16+01:00,31,5,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,2018-03-27 07:09:51+01:00,2018-03-27 19:51:50+01:00,11:01,1,True,False
3,1,2018-03-27 11:09:38+01:00,31,4,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,2018-03-27 07:09:51+01:00,2018-03-27 19:51:50+01:00,11:09,1,True,False
4,1,2018-03-27 11:10:16+01:00,31,6,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,2018-03-27 07:09:51+01:00,2018-03-27 19:51:50+01:00,11:10,1,True,False


In [9]:
# add boolean stating if it lunchtime
new_df["lunchHour"] = new_df.hourOfDay.apply(lambda time: "12:00" <= time <= "14:00")
new_df.head()

,Number,Timestamp,BikeStands,AvailableStands,WeatherTime,Temperature,WindSpeed,Clouds,Rain,Snow,Sunrise,Sunset,hourOfDay,dayOfWeek,workday,rushHour,lunchHour
0,1,2018-03-27 09:48:06+01:00,31,12,2018-03-27 10:00:00+01:00,281,9.3,75,NaN,None,2018-03-27 07:09:57+01:00,2018-03-27 19:51:46+01:00,09:48,1,True,False,False
1,1,2018-03-27 10:50:37+01:00,31,7,2018-03-27 10:30:00+01:00,281,9.8,75,NaN,None,2018-03-27 07:09:54+01:00,2018-03-27 19:51:48+01:00,10:50,1,True,False,False
2,1,2018-03-27 11:01:16+01:00,31,5,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,2018-03-27 07:09:51+01:00,2018-03-27 19:51:50+01:00,11:01,1,True,False,False
3,1,2018-03-27 11:09:38+01:00,31,4,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,2018-03-27 07:09:51+01:00,2018-03-27 19:51:50+01:00,11:09,1,True,False,False
4,1,2018-03-27 11:10:16+01:00,31,6,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,2018-03-27 07:09:51+01:00,2018-03-27 19:51:50+01:00,11:10,1,True,False,False


In [10]:
# add boolean stating if it is workhours
new_df["workHour"] = new_df.hourOfDay.apply(lambda time: "09:00" < time < "12:00" or "14:00" < time < "17:00")
new_df.head()

,Number,Timestamp,BikeStands,AvailableStands,WeatherTime,Temperature,WindSpeed,Clouds,Rain,Snow,Sunrise,Sunset,hourOfDay,dayOfWeek,workday,rushHour,lunchHour,workHour
0,1,2018-03-27 09:48:06+01:00,31,12,2018-03-27 10:00:00+01:00,281,9.3,75,NaN,None,2018-03-27 07:09:57+01:00,2018-03-27 19:51:46+01:00,09:48,1,True,False,False,True
1,1,2018-03-27 10:50:37+01:00,31,7,2018-03-27 10:30:00+01:00,281,9.8,75,NaN,None,2018-03-27 07:09:54+01:00,2018-03-27 19:51:48+01:00,10:50,1,True,False,False,True
2,1,2018-03-27 11:01:16+01:00,31,5,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,2018-03-27 07:09:51+01:00,2018-03-27 19:51:50+01:00,11:01,1,True,False,False,True
3,1,2018-03-27 11:09:38+01:00,31,4,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,2018-03-27 07:09:51+01:00,2018-03-27 19:51:50+01:00,11:09,1,True,False,False,True
4,1,2018-03-27 11:10:16+01:00,31,6,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,2018-03-27 07:09:51+01:00,2018-03-27 19:51:50+01:00,11:10,1,True,False,False,True


In [11]:
# add column containing sunrise/sunset time as hour:minute string
new_df["sunriseHour"] = new_df.Sunrise.apply(lambda time: "{0:0>2}:{1:0>2}".format(time.hour, time.minute))
new_df["sunsetHour"] = new_df.Sunset.apply(lambda time: "{0:0>2}:{1:0>2}".format(time.hour, time.minute))
new_df.head()

,Number,Timestamp,BikeStands,AvailableStands,WeatherTime,Temperature,WindSpeed,Clouds,Rain,Snow,Sunrise,Sunset,hourOfDay,dayOfWeek,workday,rushHour,lunchHour,workHour,sunriseHour,sunsetHour
0,1,2018-03-27 09:48:06+01:00,31,12,2018-03-27 10:00:00+01:00,281,9.3,75,NaN,None,2018-03-27 07:09:57+01:00,2018-03-27 19:51:46+01:00,09:48,1,True,False,False,True,07:09,19:51
1,1,2018-03-27 10:50:37+01:00,31,7,2018-03-27 10:30:00+01:00,281,9.8,75,NaN,None,2018-03-27 07:09:54+01:00,2018-03-27 19:51:48+01:00,10:50,1,True,False,False,True,07:09,19:51
2,1,2018-03-27 11:01:16+01:00,31,5,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,2018-03-27 07:09:51+01:00,2018-03-27 19:51:50+01:00,11:01,1,True,False,False,True,07:09,19:51
3,1,2018-03-27 11:09:38+01:00,31,4,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,2018-03-27 07:09:51+01:00,2018-03-27 19:51:50+01:00,11:09,1,True,False,False,True,07:09,19:51
4,1,2018-03-27 11:10:16+01:00,31,6,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,2018-03-27 07:09:51+01:00,2018-03-27 19:51:50+01:00,11:10,1,True,False,False,True,07:09,19:51


In [12]:
# add boolean stating if it has daylight
new_df["daylight"] = new_df.apply(lambda row: row["sunriseHour"] <= row["hourOfDay"] <= row["sunsetHour"], axis=1)
new_df.head()

,Number,Timestamp,BikeStands,AvailableStands,WeatherTime,Temperature,WindSpeed,Clouds,Rain,Snow,...,Sunset,hourOfDay,dayOfWeek,workday,rushHour,lunchHour,workHour,sunriseHour,sunsetHour,daylight
0,1,2018-03-27 09:48:06+01:00,31,12,2018-03-27 10:00:00+01:00,281,9.3,75,NaN,None,...,2018-03-27 19:51:46+01:00,09:48,1,True,False,False,True,07:09,19:51,True
1,1,2018-03-27 10:50:37+01:00,31,7,2018-03-27 10:30:00+01:00,281,9.8,75,NaN,None,...,2018-03-27 19:51:48+01:00,10:50,1,True,False,False,True,07:09,19:51,True
2,1,2018-03-27 11:01:16+01:00,31,5,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,...,2018-03-27 19:51:50+01:00,11:01,1,True,False,False,True,07:09,19:51,True
3,1,2018-03-27 11:09:38+01:00,31,4,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,...,2018-03-27 19:51:50+01:00,11:09,1,True,False,False,True,07:09,19:51,True
4,1,2018-03-27 11:10:16+01:00,31,6,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,...,2018-03-27 19:51:50+01:00,11:10,1,True,False,False,True,07:09,19:51,True


In [27]:
# Create classifier functions that can be used to convert numeric features into classes.
def classify_bikeStands(bikeStandsPerc):
    availability = "LOL"
    if bikeStandsPerc <= 0.33:
        availability = "little"
    elif bikeStandsPerc <= 0.66:
        availability = "moderate"
    else:
        availability = "abundant"
    return availability

def classify_temp(temp):
    temp_class = None
    if temp <= 278.15:   #cold
        temp_class = 0 
    elif temp <= 283.15: #tolerable
        temp_class = 1
    elif temp <= 288.15: #moderate
        temp_class = 2
    elif temp <= 293.15: #warm
        temp_class = 3
    else:                #hot
        temp_class = 4
    return temp_class

def classify_daytime(time, sunrise, sunset):
    daytime_class = None
    if sunrise <= time <= sunset:
        if time < "12:00":
            daytime_class = "morning"
        elif time < "17:00":
            daytime_class = "afternoon"
        else:
            daytime_class = "evening"
    else:
        if sunset < time < "24:00":
            daytime_class = "darkEvening"
        else:
            daytime_class = "night"
    return daytime_class

,Number,Timestamp,BikeStands,AvailableStands,WeatherTime,Temperature,WindSpeed,Clouds,Rain,Snow,...,Sunset,hourOfDay,dayOfWeek,workday,rushHour,lunchHour,workHour,sunriseHour,sunsetHour,daylight
0,1,2018-03-27 09:48:06+01:00,31,12,2018-03-27 10:00:00+01:00,281,9.3,75,NaN,None,...,2018-03-27 19:51:46+01:00,09:48,1,True,False,False,True,07:09,19:51,True
1,1,2018-03-27 10:50:37+01:00,31,7,2018-03-27 10:30:00+01:00,281,9.8,75,NaN,None,...,2018-03-27 19:51:48+01:00,10:50,1,True,False,False,True,07:09,19:51,True
2,1,2018-03-27 11:01:16+01:00,31,5,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,...,2018-03-27 19:51:50+01:00,11:01,1,True,False,False,True,07:09,19:51,True
3,1,2018-03-27 11:09:38+01:00,31,4,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,...,2018-03-27 19:51:50+01:00,11:09,1,True,False,False,True,07:09,19:51,True
4,1,2018-03-27 11:10:16+01:00,31,6,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,...,2018-03-27 19:51:50+01:00,11:10,1,True,False,False,True,07:09,19:51,True


In [28]:
# Convert some numeric features into categorical ones
new_df["bikeStandsPerc"]= new_df.apply(lambda row: classify_bikeStands(row.AvailableStands/row.BikeStands), axis=1)
new_df["tempClass"]= new_df.Temperature.apply(lambda temp: classify_temp(temp))
new_df["dayTime"]= new_df.apply(lambda row: classify_daytime(row.hourOfDay, row.sunriseHour, row.sunsetHour), axis=1)

## 2. Model Training

In [32]:
# Import required packages for model training
import numpy as np
import matplotlib.pyplot as plt
import pickle
from patsy import dmatrices
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

c:\users\roman\anaconda3\envs\semester_two_env\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [33]:
# the categoricla daytime column needs to be split into several boolean columns
daytimes = pd.get_dummies(new_df.dayTime, prefix='daytime').iloc[:, 1:]
new_df = pd.concat([new_df, daytimes], axis=1)
daytimes.head()

,daytime_darkEvening,daytime_evening,daytime_morning,daytime_night
0,0,0,1,0
1,0,0,1,0
2,0,0,1,0
3,0,0,1,0
4,0,0,1,0


In [35]:
new_df.head()

,Number,Timestamp,BikeStands,AvailableStands,WeatherTime,Temperature,WindSpeed,Clouds,Rain,Snow,...,sunriseHour,sunsetHour,daylight,bikeStandsPerc,tempClass,dayTime,daytime_darkEvening,daytime_evening,daytime_morning,daytime_night
0,1,2018-03-27 09:48:06+01:00,31,12,2018-03-27 10:00:00+01:00,281,9.3,75,NaN,None,...,07:09,19:51,True,moderate,1,morning,0,0,1,0
1,1,2018-03-27 10:50:37+01:00,31,7,2018-03-27 10:30:00+01:00,281,9.8,75,NaN,None,...,07:09,19:51,True,little,1,morning,0,0,1,0
2,1,2018-03-27 11:01:16+01:00,31,5,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,...,07:09,19:51,True,little,1,morning,0,0,1,0
3,1,2018-03-27 11:09:38+01:00,31,4,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,...,07:09,19:51,True,little,1,morning,0,0,1,0
4,1,2018-03-27 11:10:16+01:00,31,6,2018-03-27 11:30:00+01:00,282,8.2,75,NaN,None,...,07:09,19:51,True,little,1,morning,0,0,1,0


In [37]:
# see how the model perform by using corss validation
# Commented out as it takes ages to run
'''
for number in new_df.Number.unique():
    sub_df = new_df[new_df.Number==number]
    X = sub_df[["tempClass", "WindSpeed", "Clouds", "workday", "rushHour", "daylight", "lunchHour", "daytime_darkEvening", "daytime_evening", "daytime_morning", "daytime_night"]]
    y = sub_df["bikeStandsPerc"]
    
    rfc = RandomForestClassifier(n_estimators=100, max_features='auto', oob_score=True, random_state=1)
    scores = cross_val_score(rfc, X, y, scoring='accuracy', cv=5)
    print("Model for Station {} has Accuracy: {}".format(number, scores.mean()))
'''

Model for Station 1 has Accuracy: 0.4675714846271661
Model for Station 2 has Accuracy: 0.4042083756418215
Model for Station 3 has Accuracy: 0.6297678467930206
Model for Station 4 has Accuracy: 0.5983913129236871
Model for Station 5 has Accuracy: 0.38304914117044875
Model for Station 6 has Accuracy: 0.6351318657229198
Model for Station 7 has Accuracy: 0.5521655922902516
Model for Station 8 has Accuracy: 0.47839134835435815
Model for Station 9 has Accuracy: 0.44146037006351835
Model for Station 10 has Accuracy: 0.531311374582019
Model for Station 11 has Accuracy: 0.7155934534342985
Model for Station 12 has Accuracy: 0.5779019585714147
Model for Station 13 has Accuracy: 0.6534698715016426
Model for Station 14 has Accuracy: 0.49540274553462477
Model for Station 15 has Accuracy: 0.59947924236383
Model for Station 16 has Accuracy: 0.3618306574584058
Model for Station 17 has Accuracy: 0.5622496612380808
Model for Station 18 has Accuracy: 0.4591212071347006
Model for Station 19 has Accuracy: 0

In [57]:
# create a model for each station. This time use all data, no training and test splits
for number in new_df.Number.unique():
    # get sub df and split into descriptve and target features
    sub_df = new_df[new_df.Number==number]
    X = sub_df[["tempClass", "WindSpeed", "Clouds", "workday", "rushHour", "daylight", "lunchHour", "daytime_darkEvening", "daytime_evening", "daytime_morning", "daytime_night"]]
    y = sub_df["bikeStandsPerc"]
    
    # create model
    rfc = RandomForestClassifier(n_estimators=100, max_features='auto', oob_score=True, random_state=1)
    rfc.fit(X, y)
    
    # save the model to disk
    filename = 'models/model_station_{}.sav'.format(number)
    pickle.dump(rfc, open(filename, 'wb'))